<a href="https://colab.research.google.com/github/svanalex/HaikusFromPaintings/blob/main/SelfEvalTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Basic syllable counter

In [ ]:
import re

In [ ]:
#building a better syllable counter
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
d = cmudict.dict()

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [ ]:
import nltk
from nltk.corpus import cmudict

# Load the CMU Pronouncing Dictionary
nltk.download('cmudict')
cmu_dict = cmudict.dict()

def estimate_syllables(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word and word[0] in vowels:
        count += 1
    for i in range(1, len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

def syllable_count(word):
    word = word.lower()
    if word in cmu_dict:
        # Take the first pronunciation
        pronunciation = cmu_dict[word][0]
        syllables = len([ph for ph in pronunciation if ph[-1].isdigit()])
        return syllables
    else:
        return estimate_syllables(word)


[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [ ]:
def preprocess(sentence):
  sentence = re.sub('[^A-Za-z0-9 ]+', '', sentence)
  return sentence

In [ ]:
def line_counter(sentence):
  #need to make words from sentences
  sentence = preprocess(sentence)
  #print(sentence)
  sentence_syllables = 0
  words = sentence.split()
  for word in words:
    #print(word, syllable_count(word))
    sentence_syllables += syllable_count(word)
    #sentence_syllables += count_syllables(word)
  return sentence_syllables

##Checking for haikus using syllable counter?

In [ ]:
def syll_sentence(sentence):
  syll_line = []
  for line in sentence.split("\n"):
    print(line_counter(line))
    syll_line.append((line_counter(line), line))
  return syll_line

In [ ]:
def extract_flexible_haikus(syll_lines):
    haikus = []

    for i in range(1, len(syll_lines) - 1):
        prev_syll, prev_line = syll_lines[i - 1]
        curr_syll, curr_line = syll_lines[i]
        next_syll, next_line = syll_lines[i + 1]

        if 4 <= prev_syll <= 6 and 6 <= curr_syll <= 8 and 4 <= next_syll <= 6:
            haikus.append((f"{prev_line}\n{curr_line}\n{next_line}", [prev_syll, curr_syll, next_syll]))
            break
    return haikus if haikus else [("no/improper haiku", [0,0,0])]

In [ ]:
def score_haiku(haiku):
  score = sum([x-y for x,y in zip([5,7,5], haiku[1])])
  return score

In [ ]:
test_sentence = ".\n Here's the haiku:\nMoonlight whispers\nSilent brush strokes paint\nDreaming midnight blue\nThis haiku captures the essence of the painting's mood and style"
#test_sentence = ""
sen = syll_sentence(test_sentence)
haiku = extract_flexible_haikus(sen)
print(haiku)
print(haiku[0][0])
print(score_haiku(haiku[0]))

0
5
4
5
5
15
[('no/improper haiku', [0, 0, 0])]
no/improper haiku
17


##Llama from hugging face as an eval tool

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
#text = f"How would you rate this haiku on a scale from 1-10?: {haiku[0][0]}"
text = [f"I need to grade this haiku for syllable count on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: {haiku[0][0]}",
        f"I need to grade this haiku for its relation to nature on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: {haiku[0][0]}",
        f"I need to grade this haiku for the quality of the creativity on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: {haiku[0][0]}",
        f"I need to grade this haiku for the imagery contained within it on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: {haiku[0][0]}"
        ]


for line in text:
  inputs = tokenizer(line, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=80)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I need to grade this haiku for syllable count on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: no/improper haiku will receive a 1, a 2 will be for a haiku that is either too long or too short, a 3 will be for a haiku that is acceptable, a 4 will be for a haiku that is good, and a 5 will be for a haiku that is excellent.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I need to grade this haiku for its relation to nature on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: no/improper haiku will be graded.
This haiku is about a nature. It is a haiku.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I need to grade this haiku for the quality of the creativity on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: no/improper haiku, haiku with no creativity, haiku with creativity, haiku with good creativity, haiku with great creativity.
I'm not sure if this is a haiku. I think it's more like a couplet.
I would give this a 3/5. It's not bad, but it's not great either.
I would give this a 3/5. It's not
I need to grade this haiku for the imagery contained within it on a scale from 1-5. Treat this as a college level assignment and judge the quality appropriately: no/improper haiku, bad, mediocre, good, and excellent. Please include a short explanation of your grade.
